# DNA weaver

#### 
https://github.com/Edinburgh-Genome-Foundry/DnaWeaver/blob/master/examples/10kb_from_vendor_or_oligos.py

In [1]:
! pip install dnaweaver

In [8]:
import dnaweaver as dw
import time

cheap_dna_offer = dw.CommercialDnaOffer(
    name="CheapDNA.",
    sequence_constraints=[
        dw.NoPatternConstraint(enzyme="BsaI"),
        dw.SequenceLengthConstraint(max_length=4000),
    ],
    pricing=dw.PerBasepairPricing(0.10),
)

oligo_dna_offer = dw.CommercialDnaOffer(
    name="OliGoo",
    sequence_constraints=[
        dw.GcContentConstraint(min_gc=0.3, max_gc=0.7),
        dw.SequenceLengthConstraint(max_length=100),
    ],
    pricing=dw.PerBasepairPricing(0.07),
    memoize=True,
)

oligo_assembly_station = dw.DnaAssemblyStation(
    name="Oligo Assembly Station",
    assembly_method=dw.OligoAssemblyMethod(
        overhang_selector=dw.TmSegmentSelector(
            min_size=15, max_size=25, min_tm=50, max_tm=70
        ),
        min_segment_length=40,
        max_segment_length=200,
        sequence_constraints=[dw.SequenceLengthConstraint(max_length=1500)],
        duration=8,
        cost=30,
    ),
    supplier=oligo_dna_offer,
    coarse_grain=20,
    a_star_factor="auto",
    memoize=True,
)

assembly_station = dw.DnaAssemblyStation(
    name="Gibson Assembly Station",
    assembly_method=dw.GibsonAssemblyMethod(
        overhang_selector=dw.TmSegmentSelector(min_tm=55, max_tm=70),
        min_segment_length=100,
        max_segment_length=500,
    ),
    supplier=[cheap_dna_offer, oligo_assembly_station],
    logger="bar",
    coarse_grain=100,
    fine_grain=10,
    a_star_factor="auto",
)
print("Looking for the best assembly plan...")
t0 = time.time()
sequence = "ATGACGACGCCGTGGTCGGGAGAGTTAATTGTGGATTTATTAAATCAGCGTCCTCGCGGACAAGTCGTGTTAGATGCTGCTAATGGGGTGGATTATGAGCTGCGCTCTCAAGAGACCGGACCAATTGACATCGACGCACACGGACCCGCTCGCTGCCTGACCGGAACGGGCTCCCGTACGACGCGTTGGATTCGCCAACATCAGCGCAATACTCGCGTTTTGGGCCTGTCGCATGaccctacagttgctgacgcgcgcgaattgttacgtttaggggctttggGCCTTTTACCTAAAAATGCCGAGCCAGCGGAAATCTTCCAAGCGTTAGATCAAGCTGCCGCCGACCACATTCACACGAATGACCTTGTGCGCGACGCACCCCGTACCCCCGCTCTTCCTGAAGAAGAGGAGGCAGAACGCGTCAACGCTTTGCCAGACCAGGGCGCGGCGCGCACGGGCGCTTTGGGAGCCCAAGGTTGGCCCCACCATACTCCTGACCAACCGGCGCGCTTAGGT"
quote = assembly_station.get_quote(sequence, with_assembly_plan=True)
print(quote.assembly_step_summary())
print("Finished in %.01d seconds" % (time.time() - t0))

Looking for the best assembly plan...


Ordering plan (Gibson Assembly Station)::
  0-230: From Oligo Assembly Station - price 49.46 - lead_time 8.0
  230-519: From CheapDNA. - price 29.90 - lead_time 0.0
Price:79.36, total lead_time:8.0
Finished in 0 seconds


In [9]:
sequence

'ATGACGACGCCGTGGTCGGGAGAGTTAATTGTGGATTTATTAAATCAGCGTCCTCGCGGACAAGTCGTGTTAGATGCTGCTAATGGGGTGGATTATGAGCTGCGCTCTCAAGAGACCGGACCAATTGACATCGACGCACACGGACCCGCTCGCTGCCTGACCGGAACGGGCTCCCGTACGACGCGTTGGATTCGCCAACATCAGCGCAATACTCGCGTTTTGGGCCTGTCGCATGaccctacagttgctgacgcgcgcgaattgttacgtttaggggctttggGCCTTTTACCTAAAAATGCCGAGCCAGCGGAAATCTTCCAAGCGTTAGATCAAGCTGCCGCCGACCACATTCACACGAATGACCTTGTGCGCGACGCACCCCGTACCCCCGCTCTTCCTGAAGAAGAGGAGGCAGAACGCGTCAACGCTTTGCCAGACCAGGGCGCGGCGCGCACGGGCGCTTTGGGAGCCCAAGGTTGGCCCCACCATACTCCTGACCAACCGGCGCGCTTAGGT'

In [4]:
! pip install PyGObject

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [79 lines of output]
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/661.5 kB ? eta -:--:--
           ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 5.8 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Installing backend dependencies: started
        Installing backend dependencies: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        × Prepari